In [24]:
import json

with open('./static/evaluations/neural-experiments/1-iteration-started.json', 'r') as file:
    iteration_started_data = json.load(file)

with open('./static/evaluations/neural-experiments/1-iteration-ended.json', 'r') as file:
    iteration_finished_data = json.load(file)

started_movies = iteration_started_data['movies']

finished_selected = iteration_finished_data['selected']

def ndcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size == 0:
        return 0.0
    dcg = np.sum(r / np.log2(np.arange(2, r.size + 2)))
    idcg = np.sum(np.sort(r)[::-1] / np.log2(np.arange(2, r.size + 2)))
    return dcg / idcg if idcg > 0 else 0.0

def calculate_ndcg(recommended_items, actual_relevant_items, k=10):
    relevance_scores = [1 if item in actual_relevant_items else 0 for item in recommended_items]
    return ndcg_at_k(relevance_scores, k)

def gini_coefficient(x):
    x = np.array(x, dtype=float)
    if np.amin(x) < 0:
        x -= np.amin(x)
    x += 0.0000001
    x = np.sort(x)
    index = np.arange(1, x.shape[0] + 1)
    n = x.shape[0]
    return ((np.sum((2 * index - n - 1) * x)) / (n * np.sum(x)))


def evaluate_recommender(iteration_started_data, iteration_finished_data):
    results = {
        'EASE': {'ndcg': [], 'gini': []},
        'Combined Neural Recommender System': {'ndcg': [], 'gini': []},
        'Combined Neural Diversity Recommender System': {'ndcg': [], 'gini': []}
    }

    for algorithm, movies_data in iteration_started_data['shown'].items():
        for rec_idx, recommended_items in enumerate(movies_data):
            actual_relevant_items = iteration_finished_data['selected'][rec_idx]

            ndcg_score = calculate_ndcg(recommended_items, actual_relevant_items, k=10)
            results[algorithm]['ndcg'].append(ndcg_score)

    for algorithm in results:
        avg_ndcg = np.mean(results[algorithm]['ndcg'])
        print(f'{algorithm} - Average nDCG@10: {avg_ndcg:.4f}')

evaluate_recommender(iteration_started_data, iteration_finished_data)

EASE
[632, 1136, 341, 1129, 360, 412, 262, 226, 366, 896, 875, 1147, 1287, 1150, 358, 814, 1146, 546, 144, 715]
[875, 1147, 1287, 320, 1150, 1146, 814, 358, 1136, 546]
EASE
[299, 292, 697, 1138, 1142, 43, 1148, 496, 472, 376, 386, 1083, 1149, 629, 308, 815, 192]
[1148, 496, 299, 472, 376, 629, 1149, 1083, 386, 580]
EASE
[468, 1313, 701, 749, 125, 980, 36, 45, 1110, 1070, 268, 1174, 786, 293, 438, 1052, 706, 544]
[268, 1174, 786, 293, 438, 544, 706, 1313, 36, 1052]
EASE
[758, 962, 334, 1048, 344, 639, 56, 1135, 898, 68, 255, 0, 65, 633]
[758, 1062, 962, 334, 1048, 1135, 0, 56, 639, 344]
Combined Neural Recommender System
[632, 1136, 341, 1129, 360, 412, 262, 226, 366, 896, 875, 1147, 1287, 1150, 358, 814, 1146, 546, 144, 715]
[366, 1129, 1406, 715, 144, 360, 412, 632, 262, 1136]
Combined Neural Recommender System
[299, 292, 697, 1138, 1142, 43, 1148, 496, 472, 376, 386, 1083, 1149, 629, 308, 815, 192]
[295, 1138, 308, 815, 584, 192, 697, 43, 540, 292]
Combined Neural Recommender System


In [31]:
import pandas as pd
import json
import numpy as np

def ndcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size == 0:
        return 0.0
    dcg = np.sum(r / np.log2(np.arange(2, r.size + 2)))
    idcg = np.sum(np.sort(r)[::-1] / np.log2(np.arange(2, r.size + 2)))
    return dcg / idcg if idcg > 0 else 0.0

def calculate_ndcg(recommended_items, actual_relevant_items, k=10):
    relevance_scores = [1 if item in actual_relevant_items else 0 for item in recommended_items]
    return ndcg_at_k(relevance_scores, k)

def gini_coefficient(x):
    x = np.array(x, dtype=float)
    if np.amin(x) < 0:
        x -= np.amin(x)
    x += 0.0000001
    x = np.sort(x)
    index = np.arange(1, x.shape[0] + 1)
    n = x.shape[0]
    return ((np.sum((2 * index - n - 1) * x)) / (n * np.sum(x)))

recommendation_counts = {
    'EASE': pd.Series(dtype=int),
    'Combined Neural Recommender System': pd.Series(dtype=int),
    'Combined Neural Diversity Recommender System': pd.Series(dtype=int)
}
results = {
    'EASE': {'ndcg': [], 'gini': []},
    'Combined Neural Recommender System': {'ndcg': [], 'gini': []},
    'Combined Neural Diversity Recommender System': {'ndcg': [], 'gini': []}
}

def calculate_for_directory(directory_name):
    for iteration_number in range(1, 4):
        try:
            with open(f'{directory_name}{iteration_number}-iteration-started.json', 'r') as file:
                iteration_started_data = json.load(file)

            with open(f'{directory_name}{iteration_number}-iteration-ended.json', 'r') as file:
                iteration_finished_data = json.load(file)

            for algorithm, movies_data in iteration_started_data['shown'].items():
                for rec_idx, recommended_items in enumerate(movies_data):
                    actual_relevant_items = iteration_finished_data['selected'][rec_idx]

                    for item in recommended_items:
                        if item in recommendation_counts[algorithm]:
                            recommendation_counts[algorithm][item] += 1
                        else:
                            recommendation_counts[algorithm][item] = 1

                    ndcg_score = calculate_ndcg(recommended_items, actual_relevant_items, k=10)
                    results[algorithm]['ndcg'].append(ndcg_score)

            if iteration_number % 3 == 0:
                for algorithm in recommendation_counts:
                    item_counts = recommendation_counts[algorithm].values
                    gini_score = gini_coefficient(item_counts)
                    results[algorithm]['gini'].append(gini_score)

        except FileNotFoundError:
            print(f"Files for iteration {iteration_number} not found.")
            continue

    for algorithm in results:
        avg_ndcg = np.mean(results[algorithm]['ndcg'])
        avg_gini = np.mean(results[algorithm]['gini'])
        print(f'{algorithm} - Average nDCG@10: {avg_ndcg:.4f}, Average Gini Coefficient: {avg_gini:.4f}')



Calculations for simple scenario - 0.3 item-to-item weight and 0.2 for diversity

In [28]:
directory = './static/evaluations/neural-experiments/'
calculate_for_directory(directory)

EASE - Average nDCG@10: 0.9203, Average Gini Coefficient: 0.2172
Combined Neural Recommender System - Average nDCG@10: 0.8705, Average Gini Coefficient: 0.1561
Combined Neural Diversity Recommender System - Average nDCG@10: 0.9051, Average Gini Coefficient: 0.1561


Calculations for simple scenario - 0.8 neural weight on the first, 0.6 neural weight and 0.3 diversity weight on the second

In [29]:
directory_more_neural = './static/evaluations/more-neural/'
calculate_for_directory(directory_more_neural)

EASE - Average nDCG@10: 0.9302, Average Gini Coefficient: 0.2667
Combined Neural Recommender System - Average nDCG@10: 0.8809, Average Gini Coefficient: 0.2253
Combined Neural Diversity Recommender System - Average nDCG@10: 0.9206, Average Gini Coefficient: 0.1987


Calculations for both 0.3 neural weights and 0.4 diverse weight for the second

In [34]:
directory_more_diverse = './static/evaluations/more-diverse/'
calculate_for_directory(directory_more_diverse)

EASE - Average nDCG@10: 0.9243, Average Gini Coefficient: 0.1128
Combined Neural Recommender System - Average nDCG@10: 0.8766, Average Gini Coefficient: 0.2275
Combined Neural Diversity Recommender System - Average nDCG@10: 0.8401, Average Gini Coefficient: 0.2158


Calculations for maximal neural scenario - 1 weight for neural-only and 0.7 for diversity-tuned (diversity weight 0.299)

In [36]:
directory_max_neural = './static/evaluations/max-neural/'
calculate_for_directory(directory_max_neural)

EASE - Average nDCG@10: 0.8913, Average Gini Coefficient: 0.1570
Combined Neural Recommender System - Average nDCG@10: 0.8443, Average Gini Coefficient: 0.2458
Combined Neural Diversity Recommender System - Average nDCG@10: 0.8430, Average Gini Coefficient: 0.2495


Generally, neural algorithms suggested less relevant results. More than that, often results were quite similar for different types of users. However, these systems sometimes recommended items from the 'long tail'. With maximization of neural weight lower diverse was achieved if we consider Gini Coefficient as a relevant metric for it.